In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

Conduct feature selection with the corresponding classificaton model

Conduct featurer selection with the corresponding regression model

"""

In [124]:
# Project-wide Library calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
from sklearn.metrics import accuracy_score as AS
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



In [125]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')



# remove features with low variance
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=5)
tf=sel.fit_transform(tf)
ef=sel.fit_transform(ef)
ttf=sel.fit_transform(ttf)

tf=torch.tensor(tf)
ef=torch.tensor(ef)
ttf=torch.tensor(ttf)

# combined all data together
F = torch.cat((tf,ef,ttf))

S = torch.cat((ts,es,tts))

C = torch.cat((tc,ec,ttc))

In [126]:

"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""


'\n\nClassificaton: \n\nPatients are divided into three classes:\n\nshort-term: survival days < 10 months\nmid-term: 10 months < survival days < 15 months\nlong-term: survival days > 15 months\n\n'

In [127]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score







In [128]:
from sklearn.linear_model import RidgeClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=5)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = RidgeClassifier()
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False  True  True  True False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True]
Accuracy is:  0.6063492063492063  with a standard deviation of  0.11447295068001771


In [129]:
from sklearn.svm import SVC
# Univariate feature selection
f=SelectKBest(f_classif, k=5)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf = SVC(kernel = 'rbf')
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())



feature selected: [False False False  True  True  True False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True]
Accuracy is:  0.5119047619047619  with a standard deviation of  0.06948411565273264


In [133]:
from sklearn.tree import DecisionTreeClassifier as DT
# Univariate feature selection
f=SelectKBest(f_classif, k=7)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = DT(max_depth=15)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())


feature selected: [False False False  True  True  True False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
  True False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True]
Accuracy is:  0.7103174603174603  with a standard deviation of  0.277329570371006


In [150]:
# feature counts =  10
from sklearn.neighbors import KNeighborsClassifier as KNN
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = KNN(n_neighbors=15)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False  True  True  True False False False False False False
  True  True False False False False False False False False False False
 False  True False False False False False False False False False False
  True False False False  True False False False False False False False
 False False False False False  True False False False False False False
 False False  True]
Accuracy is:  0.5341269841269841  with a standard deviation of  0.21105889078451234


In [151]:
# feature counts = 15
from sklearn.ensemble import RandomForestClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=15)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = RandomForestClassifier(max_depth=15)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False  True  True  True False False False False False False
  True  True False False  True  True False False False False False False
 False  True False False  True False  True False False False False False
  True False False False  True False False False False False False False
 False False False False False  True False False False  True False False
 False False  True]
Accuracy is:  0.7222222222222221  with a standard deviation of  0.2215977620671906


In [152]:
# feature count = 20 
from sklearn.ensemble import ExtraTreesClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = ExtraTreesClassifier(n_estimators=25)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False  True  True  True False False False False False False
  True  True False False False False False False False False False False
 False  True False False False False False False False False False False
  True False False False  True False False False False False False False
 False False False False False  True False False False False False False
 False False  True]
Accuracy is:  0.6785714285714285  with a standard deviation of  0.24519185879438907


In [153]:
"""

Regression part

"""
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,make_scorer

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import ExtraTreesRegressor as ETR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.svm import SVR

In [154]:
# SKlearn feature extraction and regression
# Simple Linear regression
selector = SelectFromModel(estimator=LinearRegression()).fit(x_train, y_train)
print(selector.get_support())
feat=selector.transform(F)
clf=LinearRegression()
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[ True  True  True False False False False False  True  True False  True
 False False False False False False  True  True  True  True  True  True
 False False  True False False  True False False False False False False
 False  True False  True False  True False False False False False False
 False False False False False False False False False False False False
 False False False]
MSE is:  119117.06544921875  with a standard deviation of  171644.60566065914


In [155]:
# KNN regressor
selector = SelectFromModel(estimator=RFR(n_estimators = 100)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
clf=RFR(n_estimators = 100)
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[False False False False False False False False False False False False
 False False False False False False False  True False  True False False
 False  True False False  True  True  True False  True False False False
 False False False False False  True False False False False False False
 False  True False False False  True  True  True  True False False False
 False False  True]
MSE is:  46356.040158881566  with a standard deviation of  49888.99577490518


In [156]:
# Extra tree regressor
selector = SelectFromModel(estimator=ETR(n_estimators = 100)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
print(feat.shape)
clf=ETR(n_estimators = 100)
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[ True False  True False False False  True  True  True  True  True False
 False  True  True  True False  True False False False  True False  True
 False  True False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False  True False  True  True False False False
 False False  True]
(182, 19)
MSE is:  37082.36904249143  with a standard deviation of  59115.84463374816
